In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install textdistance

In [3]:
import pandas as pd
import textdistance as td
import nltk
import re

In [4]:

def fitness_score(sentence, dataframe):
  
  #Calculates how fit a sentence is based on its weighted empathy, fluency and novelty values
  
  empathy = empathy_score(sentence)
  fluency = fluency_score(sentence)
  novelty = novelty_score(sentence, dataframe)
  
  score = 2*novelty + empathy + 0.75*fluency

  return score


##Individual functions

###Empathy:

In [5]:
%%capture
!pip install transformers==2.9.0 
!pip install pytorch_lightning==0.7.5
!pip install sentencepiece

In [6]:
import argparse
import glob
import os
import json
import time
#import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import torch
#from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pandas as pd
import numpy as np

from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


INFO:transformers.file_utils:PyTorch version 1.11.0+cu113 available.
INFO:transformers.file_utils:TensorFlow version 2.8.0 available.


In [7]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    self.hparams = hparams
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.proc_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        lm_labels=lm_labels,
    )

In [8]:
#Load the tokenizer for the T5-base model
t5tokenizer = T5Tokenizer.from_pretrained('t5-base')

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpti42r0du


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model in cache at /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/t5-spiece.model from cache at /root/.cache/torch/transformers/68f1b8dbca4350743bb54b8c4169fd38cbabaad564f85a9239337a8d0342af9f.9995af32582a1a7062cb3173c118cb7b4636fa03feb967340f20fc37406f021f


In [9]:
args_dict = dict(

    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
  
)
args = argparse.Namespace(**args_dict)

In [ ]:
t5model = T5FineTuner(args)
t5model.load_state_dict(torch.load('drive/MyDrive/t5_empathy/T5_empathy_twicefinetuned.pt'))

In [11]:
def empathy_score(text):
  '''
  Computes a discrete numerical empathy score for a sentence (scale 0 to 2)
  '''
  input_ids = t5tokenizer.encode(text + '</s>', return_tensors='pt')

  output = t5model.model.generate(input_ids=input_ids, max_length=2)

  dec = [t5tokenizer.decode(ids) for ids in output]
  label = dec[0]

  if label == 'no':
    score = 0.0
  elif label == 'weak':
    score = 1.0
  else:
    score = 2.0
    
  return score/2 #normalised between 0 and 1 by dividing by the highest possible value

###Fluency:

In [12]:
%%capture
!pip install transformers
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import sys

In [13]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# Load pre-trained model (weights)
with torch.no_grad():
        gptmodel = GPT2LMHeadModel.from_pretrained('gpt2')
        gptmodel.eval()
# Load pre-trained model tokenizer (vocabulary)
gpttokenizer = GPT2Tokenizer.from_pretrained('gpt2')

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpy_1tr47b


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json in cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e05dc670366092d809a9cbcac6798e24e
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e05dc670366092d809a9cbcac6798e24e
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e05dc670366092d809a9cbcac6798e24e
INFO:transformers.configuration_utils:Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 5025

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://cdn.huggingface.co/gpt2-pytorch_model.bin in cache at /root/.cache/torch/transformers/d71fd633e58263bd5e91dd3bde9f658bafd81e11ece622be6a3c2e4d42d8fd89.778cf36f5c4e5d94c8cd9cefcf2a580c8643570eb327f0d4a1f007fab2acbdf1
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/d71fd633e58263bd5e91dd3bde9f658bafd81e11ece622be6a3c2e4d42d8fd89.778cf36f5c4e5d94c8cd9cefcf2a580c8643570eb327f0d4a1f007fab2acbdf1
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/gpt2-pytorch_model.bin from cache at /root/.cache/torch/transformers/d71fd633e58263bd5e91dd3bde9f658bafd81e11ece622be6a3c2e4d42d8fd89.778cf36f5c4e5d94c8cd9cefcf2a580c8643570eb327f0d4a1f007fab2acbdf1
INFO:transformers.modeling_utils:Weights of GPT2LMHeadModel not initialized from pretrained model: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json in cache at /root/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp40hsura6


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt in cache at /root/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /root/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f

In [14]:
#gpttokenizer.pad_token = gpttokenizer.eos_token

In [15]:

'''
#find maximum fluency score in our augmented corpus to normalise values
#here we apply the still unnormalised fluency_score function

alldf = pd.read_csv('drive/MyDrive/df0.csv') 

max_score = 0

columns = alldf.columns.tolist()  
for index, row in alldf.iterrows():
  for c in columns:
    if pd.notna(row[c]):
      score = fluency_score(row[c])
      if score > max_score:
        max_score = score
print(max_score)
'''

"\n#find maximum fluency score in our augmented corpus to normalise values\n#here we apply the still unnormalised fluency_score function\n\nalldf = pd.read_csv('drive/MyDrive/df0.csv') \n\nmax_score = 0\n\ncolumns = alldf.columns.tolist()  \nfor index, row in alldf.iterrows():\n  for c in columns:\n    if pd.notna(row[c]):\n      score = fluency_score(row[c])\n      if score > max_score:\n        max_score = score\nprint(max_score)\n"

In [16]:
def fluency_score(sentence):
  '''
  Computes the fluency score of a sentence, given by the 
  inverse of the perplexity minus a penalty for repeated tokens
  '''
  ppl = perplexity(sentence)
  penalty = repetition_penalty(sentence)

  score = (1 / ppl) - penalty
  normalised_score = score #/ 0.16  #normalise by the highest possible unnormalised fluency in our augmented corpus
  if normalised_score < 0:
    normalised_score = 0

  return round(normalised_score, 2)

In [17]:
def perplexity(sentence):
    tokenize_input = gpttokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    loss = gptmodel(tensor_input, labels=tensor_input)[0]
    return round(np.exp(loss.detach().numpy()), 2)

In [18]:
regextokenizer = RegexpTokenizer(r'\w+')
stemmer = PorterStemmer()

In [19]:
def repetition_penalty(sentence):
  '''
  Adds a penalty of 3 for each repeated (stemmed) token in
  a sentence. Returns the total penalty of the sentence.
  '''
  word_list = regextokenizer.tokenize(sentence.lower())
  filtered_words = [word for word in word_list if word not in stopwords.words('english')]
  stem_list = [stemmer.stem(word) for word in filtered_words]

  penalty = 0
  visited = []
  for w in stem_list:
    if w not in visited:
      visited.append(w)
    else: 
      penalty += 0.01

  return penalty

###Novelty:

In [20]:
%%capture
!pip install textdistance
import textdistance as td
from nltk import ngrams
import re
pd.set_option("max_colwidth", None)

In [21]:
def novelty_score(sentence, dataframe):
  '''
  Computes the weighted mean of the distances beween a sentence
  and each of the sentences in a given dataframe
  '''
  if dataframe.empty:
    score = 1.0
  else:
    d_list = compute_distances(sentence, dataframe)
    d_score = sum(d_list)
    score = d_score / len(d_list)

  return round(score, 2)

In [22]:
def compute_distances(sentence, dataframe):

  distances = []
  for index, row in dataframe.iterrows():
    df_s = dataframe['sentences'][index] #assuming the dataframe column is called 'sentences'
    distance = get_distance(df_s.lower(), sentence) 
    distances.append(distance)

  return distances

In [23]:
def get_distance(s1, s2):
  '''
  Computes a distance score between sentences calculated as the sum of the
  overlap distance between unigrams, plus the overlap distance squared over
  bigrams, plus the overlap distance cubed over trigrams, etc (up to a number
  of ngrams equal to the length of the shortest sentence)
  '''
  s1 = re.sub(r'[^\w\s]', '', s1.lower()) #preprocess
  s2 = re.sub(r'[^\w\s]', '', s2.lower())
  s1_ws = regextokenizer.tokenize(s1) #tokenize to count tokens later
  s2_ws = regextokenizer.tokenize(s2)
  max_n = len(s1_ws) if len(s1_ws) < len(s2_ws) else len(s2_ws) #the max number of bigrams is the number of tokens in the shorter sentence

  ngram_scores = []
  for i in range(1, max_n+1):
    s1grams = ngrams(s1.split(), i)
    s2grams = ngrams(s2.split(), i)
    #s1grams = [n for n in s1grams if '<MASK>' not in n]
    #s2grams = [n for n in s2grams if '<MASK>' not in n]
    ngram_scores.append((td.overlap.normalized_distance(s1grams, s2grams))**i) #we normalize the distance score to be a value between 0 and 10, before raising to i

  normalised_dis = sum(ngram_scores)/(max_n) #normalised
  return normalised_dis 

#Testing the retrieval function

In [ ]:
df = pd.read_csv('drive/MyDrive/kai.csv') # get dataset to retrieve utterances from

previous_questions = pd.DataFrame(columns=['sentences']) # start with an empty dataframe which is gradually filled with the retrieved utterances

for i in range(20): # we retrieve 20 utterances here

  maxscore = 0
  chosen = ''
  for row in df['Sad - Was this caused by a specific event/s?'].dropna(): # select a column/base utterance we want to retrieve the variations of

    if pd.notna(row):
  
       fitscore = fitness_score(row, previous_questions)
  
       if fitscore > maxscore:
         maxscore = fitscore
         chosen = row

  print(chosen)
  previous_questions = previous_questions.append({'sentences':chosen}, ignore_index=True) 
  

Oh, I am so sorry to hear this. Did something happen to make you feel this way? If so, I hope it wasn't too difficult for you.
Thank you for sharing your emotions with me, I understand it may be hard to deal with them right now. Is there anything in particular which has made you feel that way? You can be open with me. 
I'm sorry about that. Can you think of anything specific that happened to cause this? Feel free to tell me openly. 
Thank you for letting me know, I am terribly sorry you are feeling that way. Can you tell me if a specific event may have caused that? 
I'm very sorry to hear that you are sad. Did anything happen in your personal life to cause these feelings? 
I'm sorry to hear you aren't feeling well. Would you be able to tell me if there are specific situations that are making you feel this way? 
Thank you. Do you know what led to such a sad feeling, was it a specific event? 
I do understand. Do you feel that your current emotion may have been caused by a specific event 